In [1]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import pyodbc
import pandas as pd


In [2]:
def get_connection(db_config_path):
    with open(db_config_path, encoding='utf8') as db:
        database = ';'.join(db.readlines()[1:]).replace('\n', '').replace(' ', '') + ';'
    return pyodbc.connect('DRIVER={/opt/microsoft/msodbcsql18/lib64/libmsodbcsql-18.3.so.2.1};' + database + 'TrustServerCertificate=yes;')

In [3]:
con = get_connection('../config.ini')
sqlMaterial = """SELECT 
            m.ID_MATERIAL, 
            p.TIPO, 
            M.CODIGO_MATERIAL, 
            M.ITEM_SUSPENSO,
            M.MATERIAL_SUSTENTAVEL,
            c.CODIGO_CLASSE,
            c.DESCRICAO_CLASSE,
            s.CODIGO_SUBCLASSE,
            s.DESCRICAO_SUBCLASSE,
            g.CODIGO_GRUPO,
            g.DESCRICAO_GRUPO,
            d.CODIGO_DIVISAO,
            d.DESCRICAO_DIVISAO,
            s2.CODIGO_SECAO,
            s2.DESCRICAO_SECAO,
            ISNULL(STUFF((SELECT ' ' + CONCAT(c.CARACTERISTICA, ' ', CV.CARACTERISTICA_VALOR, cv.SIGLA_UNIDADE_MEDIDA)
                        FROM CATALOGO.MATERIAL_CARACTERISTICA mc
                        JOIN CATALOGO.PDM_CARACTERISTICA pc ON pc.ID_PDM_CARACTERISTICA = mc.ID_PDM_CARACTERISTICA
                        JOIN CATALOGO.CARACTERISTICAS c ON pc.ID_CARACTERISTICA = c.ID_CARACTERISTICA
                        JOIN CATALOGO.CARACTERISTICAS_VALORES cv ON cv.ID_CARACTERISTICA_VALOR = pc.ID_CARACTERISTICA_VALOR
                        WHERE mc.ID_MATERIAL = m.ID_MATERIAL
                        FOR XML PATH('')), 1, 1, ''), '') AS CARACTERISTICAS_BUSCA,
            p.NOME_MATERIAL, 
            CONCAT(p.NOME_MATERIAL, ' ', ISNULL(STUFF((SELECT ' ' + CONCAT(c.CARACTERISTICA, ' ', CV.CARACTERISTICA_VALOR, cv.SIGLA_UNIDADE_MEDIDA)
                                                    FROM CATALOGO.MATERIAL_CARACTERISTICA mc
                                                    JOIN CATALOGO.PDM_CARACTERISTICA pc ON pc.ID_PDM_CARACTERISTICA = mc.ID_PDM_CARACTERISTICA
                                                    JOIN CATALOGO.CARACTERISTICAS c ON pc.ID_CARACTERISTICA = c.ID_CARACTERISTICA
                                                    JOIN CATALOGO.CARACTERISTICAS_VALORES cv ON cv.ID_CARACTERISTICA_VALOR = pc.ID_CARACTERISTICA_VALOR
                                                    WHERE mc.ID_MATERIAL = m.ID_MATERIAL
                                                    FOR XML PATH('')), 1, 1, ''), '')) AS MATERIAL_TEXT
        FROM CATALOGO.MATERIAIS m
        JOIN CATALOGO.PDM p ON m.ID_PDM = p.ID_PDM
        LEFT JOIN CATALOGO.CLASSES c ON p.ID_CLASSE = c.ID_CLASSE
        LEFT JOIN CATALOGO.SUBCLASSES s ON c.ID_CLASSE = s.ID_CLASSE
        LEFT JOIN CATALOGO.GRUPOS g ON g.ID_GRUPO = c.ID_GRUPO
        LEFT JOIN CATALOGO.DIVISOES d ON d.ID_DIVISAO = g.ID_DIVISAO
        LEFT JOIN CATALOGO.SECOES s2 ON S2.ID_SECAO = D.ID_SECAO
        WHERE p.TIPO = 'M'
"""

In [4]:
df_materiais = pd.read_sql_query(sqlMaterial, con)
df_materiais

/tmp/ipykernel_8988/2123792220.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_materiais = pd.read_sql_query(sqlMaterial, con)


,ID_MATERIAL,TIPO,CODIGO_MATERIAL,ITEM_SUSPENSO,MATERIAL_SUSTENTAVEL,CODIGO_CLASSE,DESCRICAO_CLASSE,CODIGO_SUBCLASSE,DESCRICAO_SUBCLASSE,CODIGO_GRUPO,DESCRICAO_GRUPO,CODIGO_DIVISAO,DESCRICAO_DIVISAO,CODIGO_SECAO,DESCRICAO_SECAO,CARACTERISTICAS_BUSCA,NOME_MATERIAL,MATERIAL_TEXT
0,431550,M,600581,N,N,5905,RESISTORES,None,None,59,COMPONENTES DE EQUIPAMENTOS ELÉTRICOS E ELETRÔ...,None,None,None,None,Resistência Nominal 1k8OHMS Potência Nominal 1...,Resistor Carbono,Resistor Carbono Resistência Nominal 1k8OHMS P...
1,431551,M,600580,N,N,5905,RESISTORES,None,None,59,COMPONENTES DE EQUIPAMENTOS ELÉTRICOS E ELETRÔ...,None,None,None,None,Potência Nominal 1UN Tipo Fixo Aplicação Eletr...,Resistor Carbono,Resistor Carbono Potência Nominal 1UN Tipo Fix...
2,431552,M,600579,N,N,5905,RESISTORES,None,None,59,COMPONENTES DE EQUIPAMENTOS ELÉTRICOS E ELETRÔ...,None,None,None,None,Potência Nominal 1UN Tipo Fixo Aplicação Eletr...,Resistor Carbono,Resistor Carbono Potência Nominal 1UN Tipo Fix...
3,431553,M,600578,N,N,5905,RESISTORES,None,None,59,COMPONENTES DE EQUIPAMENTOS ELÉTRICOS E ELETRÔ...,None,None,None,None,Potência Nominal 1UN Tipo Fixo Aplicação Eletr...,Resistor Carbono,Resistor Carbono Potência Nominal 1UN Tipo Fix...
4,431554,M,600576,N,N,5905,RESISTORES,None,None,59,COMPONENTES DE EQUIPAMENTOS ELÉTRICOS E ELETRÔ...,None,None,None,None,Tipo Fixo Aplicação Eletrônica Resistência Nom...,Resistor Carbono,Resistor Carbono Tipo Fixo Aplicação Eletrônic...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253861,683446,M,377520,S,N,6145,FIOS E CABOS ELÉTRICOS,None,None,61,CONDUTORES ELÉTRICOS E EQUIPAMENTOS PARA GERAÇ...,None,None,None,None,Aplicação Uso Em Vídeo Características Adicion...,Cabo De Vídeo,Cabo De Vídeo Aplicação Uso Em Vídeo Caracterí...
253862,683447,M,377519,S,N,6145,FIOS E CABOS ELÉTRICOS,None,None,61,CONDUTORES ELÉTRICOS E EQUIPAMENTOS PARA GERAÇ...,None,None,None,None,Aplicação Uso Em Vídeo N° De Cabos 3UN Tipo An...,Cabo De Vídeo,Cabo De Vídeo Aplicação Uso Em Vídeo N° De Cab...
253863,683780,M,612103,N,N,6540,"EQUIPAMENTOS, INSTRUMENTOS E SUPRIMENTOS OFTAL...",None,None,65,"EQUIPAMENTOS E ARTIGOS PARA USO MÉDICO, DENTÁR...",None,None,None,None,Tipo Grande Ocular Modelo De Não Contato Aplic...,Sistema De Lentes Para Equipamento Oftalmológico,Sistema De Lentes Para Equipamento Oftalmológi...
253864,683781,M,229763,N,N,6540,"EQUIPAMENTOS, INSTRUMENTOS E SUPRIMENTOS OFTAL...",None,None,65,"EQUIPAMENTOS E ARTIGOS PARA USO MÉDICO, DENTÁR...",None,None,None,None,Material Cristal Características Adicionais Au...,Sistema De Lentes Para Equipamento Oftalmológico,Sistema De Lentes Para Equipamento Oftalmológi...


In [5]:
pd.set_option('display.max_colwidth', None)
df_materiais['MATERIAL_TEXT']

0                                                                                                                            Resistor Carbono Resistência Nominal 1k8OHMS Potência Nominal 1UN Tipo Fixo Aplicação Eletrônica
1                                                                                                                             Resistor Carbono Potência Nominal 1UN Tipo Fixo Aplicação Eletrônica Resistência Nominal 1k5OHM
2                                                                                                                             Resistor Carbono Potência Nominal 1UN Tipo Fixo Aplicação Eletrônica Resistência Nominal 1k2OHM
3                                                                                                                                Resistor Carbono Potência Nominal 1UN Tipo Fixo Aplicação Eletrônica Resistência Nominal 27W
4                                                                                                               

In [6]:
pd.set_option('display.max_colwidth', None)
df_materiais['MATERIAL_TEXT']

0                                                                                                                            Resistor Carbono Resistência Nominal 1k8OHMS Potência Nominal 1UN Tipo Fixo Aplicação Eletrônica
1                                                                                                                             Resistor Carbono Potência Nominal 1UN Tipo Fixo Aplicação Eletrônica Resistência Nominal 1k5OHM
2                                                                                                                             Resistor Carbono Potência Nominal 1UN Tipo Fixo Aplicação Eletrônica Resistência Nominal 1k2OHM
3                                                                                                                                Resistor Carbono Potência Nominal 1UN Tipo Fixo Aplicação Eletrônica Resistência Nominal 27W
4                                                                                                               

In [7]:
df_materiais.isnull().sum()

ID_MATERIAL                   0
TIPO                          0
CODIGO_MATERIAL               0
ITEM_SUSPENSO                 0
MATERIAL_SUSTENTAVEL          0
CODIGO_CLASSE                 0
DESCRICAO_CLASSE              0
CODIGO_SUBCLASSE         247773
DESCRICAO_SUBCLASSE      247773
CODIGO_GRUPO                  0
DESCRICAO_GRUPO               0
CODIGO_DIVISAO           253866
DESCRICAO_DIVISAO        253866
CODIGO_SECAO             253866
DESCRICAO_SECAO          253866
CARACTERISTICAS_BUSCA         0
NOME_MATERIAL                 0
MATERIAL_TEXT                 0
dtype: int64

In [8]:
es = Elasticsearch(
    "http://localhost:9200",
    request_timeout=999999999,
)

es.info()

ObjectApiResponse({'name': 'melquicarvalho', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'VxyIZW2KQg-9GWvUtR9Qsw', 'version': {'number': '8.12.2', 'build_flavor': 'default', 'build_type': 'deb', 'build_hash': '48a287ab9497e852de30327444b0809e55d46466', 'build_date': '2024-02-19T10:04:32.774273190Z', 'build_snapshot': False, 'lucene_version': '9.9.2', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [9]:
indices = es.cat.indices(h='index', format='json')
nomes_indices = [indice['index'] for indice in indices]
nomes_indices

['index_t_nf', 'index_s']

In [10]:
def doc_generator(index_name, df):
  for idx, row in df.iterrows():
    yield {
        "_index": index_name,
        "_id": idx,
        "_source": {
            "id_material": row['ID_MATERIAL'],
            "tipo": row['TIPO'],
            "codigo_material": row['CODIGO_MATERIAL'],
            "item_suspenso": row['ITEM_SUSPENSO'],
            "material_sustentavel": row['MATERIAL_SUSTENTAVEL'],
            "codigo_classe": row['CODIGO_CLASSE'],
            "descricao_classe": row['DESCRICAO_CLASSE'],
            "codigo_subclasse": row['CODIGO_SUBCLASSE'],
            "descricao_subclasse": row['DESCRICAO_SUBCLASSE'],
            "codigo_grupo": row['CODIGO_GRUPO'],
            "descricao_grupo": row['DESCRICAO_GRUPO'],
            "codigo_divisao": row['CODIGO_DIVISAO'],
            "descricao_divisao": row['DESCRICAO_DIVISAO'],
            "codigo_secao": row['CODIGO_SECAO'],
            "descricao_secao": row['DESCRICAO_SECAO'],
            "caracteristica_busca": row['CARACTERISTICAS_BUSCA'],
            "nome_material": row['NOME_MATERIAL'],
            "material_text": row['MATERIAL_TEXT'],
        }
    }

In [11]:
INDEX_NAME = "index_t"
documentos_nf_index_generator = doc_generator(INDEX_NAME, df_materiais)

In [12]:
es.options(ignore_status=[400,404]).indices.delete(index=INDEX_NAME)

body = {
    "settings": {
        "number_of_shards": 3,
        "number_of_replicas": 1,
        "analysis": {
            "analyzer": {
                "standard_asciifolding": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": [ "lowercase", "asciifolding" ]
                }
            }
        }
  },
    
    'mappings': {
        'properties': {
            'tipo': {'type': 'keyword' },
            'id_material': {'type': 'long' },
            'codigo_material': {'type': 'long' },
            'item_suspenso': {'type': 'keyword' },
            'material_sustentavel': {'type': 'keyword' },
            'codigo_classe': {'type': 'keyword' },
            'descricao_classe': {'type': 'text' },
            'codigo_subclasse': {'type': 'keyword' },
            'descricao_subclasse': {'type': 'text' },
            'codigo_grupo': {'type': 'keyword' },
            'descricao_grupo': {'type': 'text' },
            'codigo_divisao': {'type': 'keyword' },
            'descricao_divisao': {'type': 'text' },
            'codigo_secao': {'type': 'keyword' },
            'descricao_secao': {'type': 'text' },
            'caracteristica_busca': {'type': 'text' },
            'nome_material': {'type': 'text' },
            'material_text': {'type': 'text', 'analyzer': 'brazilian', "fields": {
                                                                                    "acentuado": { 
                                                                                    "type":       "text",
                                                                                    "analyzer":   "standard_asciifolding"
                                                                                    }
                                                                                }
                            },
        }
    }
}
es.indices.create(index=INDEX_NAME, body=body)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'index_t'})

In [13]:
indices = es.cat.indices(h='index', format='json')
nomes_indices = [indice['index'] for indice in indices]
nomes_indices

['index_t_nf', 'index_s', 'index_t']

In [14]:
try:
    response = helpers.bulk(es, documentos_nf_index_generator)

    if response[1]:
        errors = response[1]
        for error in errors:
            print("Falha de indexação:", error)
except helpers.BulkIndexError as e:
    for err in e.errors:
        print("Falha de indexação:", err)

In [15]:
doc_id = 2
resp = es.get(index=INDEX_NAME, id=doc_id)
print(resp)

{'_index': 'index_t', '_id': '2', '_version': 1, '_seq_no': 1, '_primary_term': 1, 'found': True, '_source': {'id_material': 431552, 'tipo': 'M', 'codigo_material': '600579', 'item_suspenso': 'N', 'material_sustentavel': 'N', 'codigo_classe': '5905', 'descricao_classe': 'RESISTORES', 'codigo_subclasse': None, 'descricao_subclasse': None, 'codigo_grupo': '59', 'descricao_grupo': 'COMPONENTES DE EQUIPAMENTOS ELÉTRICOS E ELETRÔNICOS', 'codigo_divisao': None, 'descricao_divisao': None, 'codigo_secao': None, 'descricao_secao': None, 'caracteristica_busca': 'Potência Nominal 1UN Tipo Fixo Aplicação Eletrônica Resistência Nominal 1k2OHM', 'nome_material': 'Resistor Carbono', 'material_text': 'Resistor Carbono Potência Nominal 1UN Tipo Fixo Aplicação Eletrônica Resistência Nominal 1k2OHM'}}


In [16]:
query = {
    "query": {
        "match": {
            "material_text": "Fabricante Detroit Diesel"
        }
    },
    "size": 16,
}

resp = es.search(index=INDEX_NAME, body=query)
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print(hit["_source"]['material_text'])

Got 4170 Hits:
Filtro Combustível Tipo Combustível Óleo Diesel Aplicação 1 Veiculo Blindado 113br Características Adicionais Secundário Referência Fabricante Detroit Diesel 23518482
Filtro Combustível Tipo Combustível Óleo Diesel Características Adicionais Primário Aplicação 1 Veiculo Blindado 113br Referência Fabricante Detroit Diesel 23517471
Filtro Combustível Tipo Combustível Óleo Diesel Características Adicionais Primário Aplicação 1 Veiculo Blindado 113br Referência Fabricante Detroit Diesel 23518482
Filtro Combustível Tipo Combustível Diesel Referência Fabricante Nsn 2910-00-781-1354 Aplicação 1 Motor Detroit Diesel 6v 5063-539a
Elemento Filtrante Material Fibra Aplicação Motor Detroit Diesel 6v 5063-539a Referência Fabricante Nsn 4730-00-745-7741
Bico Bomba Injetora Combustível Tipo Uso Motor A Diesel Aplicação Bomba N80 Do  Motor Detroit 8v71t Referência Fabricante 522 9256
Escova Contato Elétrico Material Metal Aplicação Motor Detroit Diesel 6v 5063-539a Referência Fabricante

In [17]:
query = {
    "query": {
        "match": {
            "material_text": "Eixo Primario Aplicação"
        }
    },
    "size": 16,
}

resp = es.search(index=INDEX_NAME, body=query)
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print(hit["_source"]['material_text'])

Got 10000 Hits:
Rolamento Tipo Uso Eixo Primário Aplicação Trator Massey Ferguson 292 Referência 886909
Rolamento Tipo Uso Eixo Primário Grande Aplicação Veículo Fiat Elba Weekend Referência 0001533
Retentor Roda - Veículo Automotivo Aplicação Trator Massey Ferguson 292 Tipo Uso Eixo Primário Referência 883935
Peça Mecânica/Elétrica - Veículo Automotivo Tipo 3 Flange Do Eixo Primário Uso Caixa De Descida Aplicação Veículo Blindado Cascavel / Urutu Referência 6 2282-305-014
Rolamento Tipo Uso Eixo Primário Aplicação Ônibus Mercedez-Benz, Modelo Om355/6 - Plataforma Tipo Esfera Com Ranhura Externa Para Presilha Características Adicionais Blindagem Externa Em Um Dos Lados
Peça Mecânica/Elétrica - Veículo Automotivo Aplicação Viatura Blindada Engesa Cascavel Modelo M2 Uso Caixa De Marcha Tipo 6 Chaveta Eixo Primario Referência 13 2680-301-096
Diglicina Tipo Teste Diagnóstico Aplicação Diagnóstico Do Hiperparatireoidismo Primário
Motor Primario De Aeronave Nome Agenda
Motor Primario De Miss

In [18]:
query_phrase = {
    "query": {
        "match_phrase": {
            "material_text": "Fabricante Detroit Diesel"
        }
    },
    "size": 8
}
resp = es.search(index=INDEX_NAME, body=query_phrase)
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print(hit["_source"]['material_text'])

Got 3 Hits:
Filtro Combustível Tipo Combustível Óleo Diesel Aplicação 1 Veiculo Blindado 113br Características Adicionais Secundário Referência Fabricante Detroit Diesel 23518482
Filtro Combustível Tipo Combustível Óleo Diesel Características Adicionais Primário Aplicação 1 Veiculo Blindado 113br Referência Fabricante Detroit Diesel 23517471
Filtro Combustível Tipo Combustível Óleo Diesel Características Adicionais Primário Aplicação 1 Veiculo Blindado 113br Referência Fabricante Detroit Diesel 23518482


In [19]:
query_filter = {
    "query": {
        "bool": {
        "must": [
            {
            "match": {
                "material_text": "detroit diesel"
            }
            }
        ],
        "filter": [
            {
                "term": {
                    "item_suspenso": "S"
                },
                "term": {
                    "codigo_classe": "6115"
                }
            }
        ]
    }
  },
  "sort": [
  ]
}

resp = es.search(index=INDEX_NAME, body=query_filter)
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print(hit["_source"]['material_text'])

Got 50 Hits:
Grupo Diesel Gerador Nome Agenda
Unidade A Diesel Com Gerador De Energia Eletrica Nome Agenda
Grupo Diesel Gerador Potência 8MB Tensão Saída 120/240V
Grupo Diesel Gerador Frequência 60MM Tensão Saída 220/127V Aplicação Sistema De Emergência Fator Potência 0,80PER Potência 290/260KVA Características Adicionais 2 Grupos Geradores Diesel, Paralelo
Induzido Do Gerador Eletrico Uso Motor Partida Aplicação Veiculo Land Rover Defender 4x4 Diesel Referência V158608
Grupo Diesel Gerador Quantidade Fases Trifásico Tipo Comando Automático E Manual Potência 75 KvaKVA Fator Potência 0,8MG/ML Frequência 60MM Componentes Tanque De 60 Litros , Disjuntor Maual E Chave De T Características Adicionais Elétrico, Potência 75 Kva,Tipo Motor Diesel, 3 Cil
Grupo Motor-Gerador Tipo Do Motor De Acionamento A Diesel Número De Fases 1UN Potência Motor 13 Potência Nominal 6MM
Induzido Do Gerador Eletrico Uso Motor Partida Aplicação Veículo Agrale Marruá 4 X 4 Diesel Am Referência 6011.001.246.00.9
Gru

In [20]:
query_fields = {
 "query": {
    "multi_match": {
      "query": "material de informática",
      "fields": ["caracteristica_busca^3", "material_text"],
      "type": "cross_fields",
    }
  }
}

resp = es.search(index=INDEX_NAME, body=query_fields)
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print(hit["_source"]['caracteristica_busca'])
    print(hit["_source"]['material_text'])
    print('---------------------')



Got 10000 Hits:
Quantidade Saída 4MG Aplicação Equipamento De Informática
Divisor Rf Quantidade Saída 4MG Aplicação Equipamento De Informática
---------------------
Aplicação Equipamento De Informática Quantidade Saída 2MG
Divisor Rf Aplicação Equipamento De Informática Quantidade Saída 2MG
---------------------
Aplicação Equipamento De Informática Quantidade Saída 3UN
Divisor Rf Aplicação Equipamento De Informática Quantidade Saída 3UN
---------------------
Aplicação Cabeamento Estruturado De Rede De Informática Tipo Rj 45 Macho Material Plástico Rígido
Conector Aplicação Cabeamento Estruturado De Rede De Informática Tipo Rj 45 Macho Material Plástico Rígido
---------------------
Aplicação Equipamento De Informática Modelo Computador Notebook Referência 14"
Bolsa Transporte Aplicação Equipamento De Informática Modelo Computador Notebook Referência 14"
---------------------
Material Polipropileno Espessura 22 Micra Tipo Duplo Largura 35W Aplicação Componentes De Informática
Filme Plást

In [21]:
query_pesos = {
  "query": {
    "multi_match": {
      "query": "material de Informática",
      "fields": ["caracteristica_busca", "material_text^3"],
      "type": "cross_fields",
    }
  }
}

resp = es.search(index=INDEX_NAME, body=query_pesos)
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print(hit["_source"]["caracteristica_busca"])
    print(hit["_source"]["material_text"])
    print("__________________")


Got 10000 Hits:
Nome Agenda
Mobiliário De Informática Nome Agenda
__________________
Gramatura 120MS Comprimento 297 Largura 210MB Características Adicionais Impressão Policromia Material Papel Couchê Aplicação Informativo Tipo Boletim Informativo
Impresso Padronizado Gramatura 120MS Comprimento 297 Largura 210MB Características Adicionais Impressão Policromia Material Papel Couchê Aplicação Informativo Tipo Boletim Informativo
__________________
Comprimento 297 Largura 210MB Material Papel Couchê Gramatura 115MM Aplicação Informativo Tipo Boletim Informativo Características Adicionais Impressão Policromia, 8 Folhas, Formato 8
Impresso Padronizado Comprimento 297 Largura 210MB Material Papel Couchê Gramatura 115MM Aplicação Informativo Tipo Boletim Informativo Características Adicionais Impressão Policromia, 8 Folhas, Formato 8
__________________
Nome Agenda
Rede Informática - Peça / Acessório Nome Agenda
__________________
Comprimento 297 Largura 210MB Material Papel Couchê Gramatura 

In [22]:
query_must = {
  "query": {
    "bool": {
            "must": [
                {"match": {"material_text": "envio de produtos"}},
            ],
    }
  }, "size": 8
}

resp = es.search(index=INDEX_NAME, body=query_must)
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print(hit["_source"]['material_text'])


Got 369 Hits:
Módulo Eletrônico Modelo Albatross Master Uso Enviar Comando Para Módulos Escravos Aplicação Placa Shield Interface Arduino Características Adicionais Armazenar Enviar Comandos Via Albatross Cabo Ir Tipo Controle Infravermelho
Aparelho Fac-Símile Tipo Papel Impressão Térmico / Bobina Tensão Alimentação 110MM Características Adicionais Corte Automático, Impressão Recibo De Envio
Envelope Para Convite Cor Branca Comprimento 225KVA Gramatura 120MS Material Papel Apergaminhado Altura 225KVA Aplicação Enviar Convite
Produto De Confeitaria Nome Agenda
Moinho Para Industria De Produtos Quimicos Material Corpo Aço Carbono Finalidade Maquinas Para Industria De Produtos Quiimicos E Fa
Embalagem Produto Farmacêutico Nome Agenda
Mangueira Produto Quimico Nome Agenda
Prensa Produto Alimentício Nome Agenda


In [23]:
query_must_analyzer_asciifolding = {
  "query": {
    "bool": {
            "must": [
                {"match": {"material_text.acentuado": "informática"}},
            ],
        }
  },"size": 20
}

resp = es.search(index=INDEX_NAME, body=query_must_analyzer_asciifolding)
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print(hit["_source"])


Got 429 Hits:
{'id_material': 564156, 'tipo': 'M', 'codigo_material': '54194', 'item_suspenso': 'S', 'material_sustentavel': 'N', 'codigo_classe': '7195', 'descricao_classe': 'MOBILIÁRIOS DIVERSOS E ACESSÓRIOS', 'codigo_subclasse': None, 'descricao_subclasse': None, 'codigo_grupo': '71', 'descricao_grupo': 'MOBILIÁRIOS', 'codigo_divisao': None, 'descricao_divisao': None, 'codigo_secao': None, 'descricao_secao': None, 'caracteristica_busca': 'Nome Agenda', 'nome_material': 'Mobiliário De Informática', 'material_text': 'Mobiliário De Informática Nome Agenda'}
{'id_material': 579545, 'tipo': 'M', 'codigo_material': '139343', 'item_suspenso': 'S', 'material_sustentavel': 'N', 'codigo_classe': '7080', 'descricao_classe': 'PEÇAS, ACESSÓRIOS E FERRAMENTAS PARA REDES DE TIC', 'codigo_subclasse': None, 'descricao_subclasse': None, 'codigo_grupo': '70', 'descricao_grupo': 'INFORMÁTICA - EQUIPAMENTOS,  PEÇAS, ACESSÓRIOS E SUPRIMENTOSDE TIC', 'codigo_divisao': None, 'descricao_divisao': None, 'cod

In [24]:
queryShould = {
  "query": {
    "bool": {
      "should": [
        { "match": { "material_text": "jeans" }},
        { "match": { "material_text": "moletom" }}
      ],
    }
  },
}

resp = es.search(index=INDEX_NAME, body=queryShould)
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print(hit["_source"]['material_text'])

Got 131 Hits:
Tecido Material 100% Algodão Cor Variada Largura 1,80M Características Adicionais Gramatura  240g/M Aplicação Confecção De Roupas Tipo Moletom Sem Felpar
Tecido Material 100% Algodão Cor Variada Tipo Moletom Felpado Gramatura 240g/MG/M2 Aplicação Confecção De Roupas Características Adicionais Kilo Largura 1,80m AproximadaCM
Tecido Material 100% Algodão Aplicação Vestuário Confeccão De Roupas Cor Variada Largura 1,80M Tipo Moletom Felpado Características Adicionais Gramatura  240g/M
Calça Tamanho Variado Aplicação Uniforme Material Jeans Modelo Masculino
Calça Tamanho Variado Aplicação Uniforme Material Jeans Modelo Feminino
Componente Arco Ortodôntico Extrabucal Material Jeans Tipo Para Tracionador Tamanho Médio
Bolsa Para Arco - Uso Ortodôntico Tamanho Único Material Jeans Cor Cinza
Bolsa Para Arco - Uso Ortodôntico Tamanho Único Cor Rosa Material Jeans
Bolsa Para Arco - Uso Ortodôntico Tamanho Único Material Jeans Cor Laranja
Bolsa Para Arco - Uso Ortodôntico Tamanho Ún

In [25]:
queryAggs = {
  "query": {
    "bool": {
      "must": [
        { "match": { "material_text": "jeans" }}
      ],
    }
  },
  "aggs": {
    "grupos": {
      "terms": { "field": "codigo_material.keyword" }
    },
  }
}


resp_aggregation = es.search(index=INDEX_NAME, body=queryAggs)
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print(hit["_source"])

Got 131 Hits:
{'id_material': 648925, 'tipo': 'M', 'codigo_material': '603863', 'item_suspenso': 'N', 'material_sustentavel': 'N', 'codigo_classe': '8305', 'descricao_classe': 'TECIDOS', 'codigo_subclasse': None, 'descricao_subclasse': None, 'codigo_grupo': '83', 'descricao_grupo': 'TECIDOS, COUROS, PELES, AVIAMENTOS, BARRACAS E BANDEIRAS', 'codigo_divisao': None, 'descricao_divisao': None, 'codigo_secao': None, 'descricao_secao': None, 'caracteristica_busca': 'Material 100% Algodão Cor Variada Largura 1,80M Características Adicionais Gramatura  240g/M Aplicação Confecção De Roupas Tipo Moletom Sem Felpar', 'nome_material': 'Tecido', 'material_text': 'Tecido Material 100% Algodão Cor Variada Largura 1,80M Características Adicionais Gramatura  240g/M Aplicação Confecção De Roupas Tipo Moletom Sem Felpar'}
{'id_material': 648923, 'tipo': 'M', 'codigo_material': '603926', 'item_suspenso': 'N', 'material_sustentavel': 'N', 'codigo_classe': '8305', 'descricao_classe': 'TECIDOS', 'codigo_sub

In [26]:
queryAggs = {
    "query": {
    "bool": {
      "must": [
        { "match": { "material_text": "jeans" }}
      ],
      "must_not": [
        {"match": {"material_text": "modelo masculino"}}
      ],
    }
  },
    "aggs": {
        "name": {
            "terms": {
                "field": "item_suspenso"
            }
        }
    }  
}

resp = es.search(index=INDEX_NAME, body=queryAggs)
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print(hit["_source"])

Got 10 Hits:
{'id_material': 483106, 'tipo': 'M', 'codigo_material': '220289', 'item_suspenso': 'N', 'material_sustentavel': 'N', 'codigo_classe': '6520', 'descricao_classe': 'INSTRUMENTOS, EQUIPAMENTOS E SUPRIMENTOS DENTÁRIOS', 'codigo_subclasse': None, 'descricao_subclasse': None, 'codigo_grupo': '65', 'descricao_grupo': 'EQUIPAMENTOS E ARTIGOS PARA USO MÉDICO, DENTÁRIO E VETERINÁ-RIO', 'codigo_divisao': None, 'descricao_divisao': None, 'codigo_secao': None, 'descricao_secao': None, 'caracteristica_busca': 'Material Jeans Tipo Para Tracionador Tamanho Médio', 'nome_material': 'Componente Arco Ortodôntico Extrabucal', 'material_text': 'Componente Arco Ortodôntico Extrabucal Material Jeans Tipo Para Tracionador Tamanho Médio'}
{'id_material': 666935, 'tipo': 'M', 'codigo_material': '319290', 'item_suspenso': 'N', 'material_sustentavel': 'N', 'codigo_classe': '6520', 'descricao_classe': 'INSTRUMENTOS, EQUIPAMENTOS E SUPRIMENTOS DENTÁRIOS', 'codigo_subclasse': None, 'descricao_subclasse'

In [27]:
queryAggs = {
"query": {
    "bool": {
      "should": [
        { "match": { "material_text": "jeans" }},
        { "match": { "material_text": "moletom" }}
      ],
    }
  },
 "aggs": {
        "descricao_grupo_agg": {
            "terms": {"field": "codigo_grupo"}
        }
    }
}

resp_aggregation = es.search(index=INDEX_NAME, body=queryAggs)
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print(hit["_source"])

Got 10 Hits:
{'id_material': 483106, 'tipo': 'M', 'codigo_material': '220289', 'item_suspenso': 'N', 'material_sustentavel': 'N', 'codigo_classe': '6520', 'descricao_classe': 'INSTRUMENTOS, EQUIPAMENTOS E SUPRIMENTOS DENTÁRIOS', 'codigo_subclasse': None, 'descricao_subclasse': None, 'codigo_grupo': '65', 'descricao_grupo': 'EQUIPAMENTOS E ARTIGOS PARA USO MÉDICO, DENTÁRIO E VETERINÁ-RIO', 'codigo_divisao': None, 'descricao_divisao': None, 'codigo_secao': None, 'descricao_secao': None, 'caracteristica_busca': 'Material Jeans Tipo Para Tracionador Tamanho Médio', 'nome_material': 'Componente Arco Ortodôntico Extrabucal', 'material_text': 'Componente Arco Ortodôntico Extrabucal Material Jeans Tipo Para Tracionador Tamanho Médio'}
{'id_material': 666935, 'tipo': 'M', 'codigo_material': '319290', 'item_suspenso': 'N', 'material_sustentavel': 'N', 'codigo_classe': '6520', 'descricao_classe': 'INSTRUMENTOS, EQUIPAMENTOS E SUPRIMENTOS DENTÁRIOS', 'codigo_subclasse': None, 'descricao_subclasse'